In [1]:
import traceback
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import torch

# вернемся к разложению градиента функции потерь по весам на модельную часть и часть зависящую от функции потерь

$$
\nabla_w L_{oss} = J_{model}^T \,\cdot\, \nabla_{\hat y} L_{oss}
$$

## матрица Якоби определяется только моделью - это линейная часть зависимости вектора ответов $\hat y$ от вектора переметров $w$, включая интерсепт

$$
d\hat y \equiv J_{model} \cdot dw \,\,\,\Rightarrow\,\,\, J^{model}_{ji} \equiv \left(\frac{\partial\hat y_j}{\partial w_i}\right)
$$

## градиент же функции потерь по ответам модели определяется только формулой функции потерь и не зависит от весов и модели

$$
\nabla_{\hat y} L_{oss} = \frac{2}{n}\, (\hat y - y)
$$

**если функция потерь - MSE, то этот градиент в пространстве ответов пропорционален ошибке ответа на разметке, поэтому формула итеративного обновления весов в GD называется алгоритмом обратного распространения ошибки: в прямом проходе мы вычисляем новые ошибки - используя текущие веса модели, в обратном проходе по ошибкам мы линейно обновляем веса:**

$$
w' = w - \frac{2\delta}{n}\; J_{model}^T \,\cdot\, (\hat y - y)
$$

# имея шаговые инкременты весов и ответов, можно ли параллельно оценивать градиент без вывода аналитической формулы?

**NB!**: PyTorch tensors can remember where they come from, in terms of the operations and parent tensors that originated them, and they can automatically provide the chain of derivatives of such operations with respect to their inputs. This means we won’t need to derive our model by hand: given a forward expression, no matter how nested, PyTorch will automatically provide the gradient of that expression with respect to its input parameters.

## поэкспериментируем с `autograd`

In [2]:
x_vector = torch.tensor([1.0,1.0], requires_grad=True)
x_vector, x_vector.grad

(tensor([1., 1.], requires_grad=True), None)

In [3]:
y_scalar = (x_vector**2).sum()
y_scalar, x_vector.grad

(tensor(2., grad_fn=<SumBackward0>), None)

In [4]:
y_scalar.backward()
x_vector.grad

tensor([2., 2.])

In [5]:
try:
    y_scalar.backward()
    print(x_vector.grad)
except:
    error_msg = traceback.format_exc()
    print('-'*80)
    print('FYI:', error_msg.split('RuntimeError:')[-1].strip())
    print('-'*80)
    
    x_vector.grad = None
    y_scalar = (x_vector**2).sum()
    y_scalar.backward()
    print('going via except:', x_vector.grad)

--------------------------------------------------------------------------------
FYI: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.
--------------------------------------------------------------------------------
going via except: tensor([2., 2.])


## а теперь если функция не скаляр, а вектор?

In [6]:
X = torch.tensor([
    [1.,2.],
    [3.,4.],
    [5.,6.],
    [7.,8.]
])
X

tensor([[1., 2.],
        [3., 4.],
        [5., 6.],
        [7., 8.]])

In [7]:
x_vector.grad = None
y_vector = X @ x_vector
y_vector, x_vector.grad

(tensor([ 3.,  7., 11., 15.], grad_fn=<MvBackward0>), None)

In [8]:
try:
    y_vector.backward()
    print(x_vector.grad)
except:
    error_msg = traceback.format_exc()
    print('-'*80)
    print('FYI:', error_msg.split('RuntimeError:')[-1].strip())
    print('-'*80)

    J = []
    for idx in range(y_vector.shape[0]):
        x_vector.grad = None
        y_vector = X @ x_vector
        y_vector[idx].backward()
        J.append(x_vector.grad)
    J = torch.stack(J)
    print('going via except:')
    print(J)

--------------------------------------------------------------------------------
FYI: grad can be implicitly created only for scalar outputs
--------------------------------------------------------------------------------
going via except:
tensor([[1., 2.],
        [3., 4.],
        [5., 6.],
        [7., 8.]])


In [9]:
if (X != J).sum() == 0:
    print('Матрица Якоби найдена верно!')
else:
    print('Матрица Якоби неверна!')

Матрица Якоби найдена верно!


**заметим, что сообщение об ошибке (а также`MvBackward` вместо `SumBackward` у векторной функции `tensor([ 3.,  7., 11., 15.], grad_fn=<MvBackward0>)`) неявно указывает, что эксплицитно и автоград как-то может находить градиент для векторной функции - надежда избежать повторных вычислений каждой компоненты функции все же есть**

## повторим упражнение с GD без аналитической формулы градиента, но с `autograd`

### data

In [10]:
t_c = torch.tensor([0.5, 14.0, 15.0, 28.0, 11.0, 8.0,
                    3.0, -4.0, 6.0, 13.0, 21.0])
t_u = torch.tensor([35.7, 55.9, 58.2, 81.9, 56.3, 48.9,
                    33.9, 21.8, 48.4, 60.4, 68.4])
t_un = 0.1 * t_u

### model

In [11]:
def model(t_u, params):
    X = torch.stack([torch.ones(t_u.shape[0]), t_u]).T
    return X @ params

### loss

In [12]:
def loss_fn(t_p, t_c):
    squared_diffs = (t_p - t_c)**2
    return squared_diffs.mean()

### starting point

In [13]:
params = torch.tensor([0.0, 1.0], requires_grad=True)
print('b =', params[0], '\nw =', params[1])
params

b = tensor(0., grad_fn=<SelectBackward0>) 
w = tensor(1., grad_fn=<SelectBackward0>)


tensor([0., 1.], requires_grad=True)

In [14]:
print(params.grad)

None


### starting loss & autograd

In [15]:
loss = loss_fn(model(t_u, params), t_c)
loss.backward()

params.grad

tensor([  82.6000, 4517.2969])

### NB!: для каждой новой точки $(b,w)$ надо руками завойдить градиент в тензоре параметров перед вычислением функции потерь в ней и вызова autograd-метода `tensor.backward()`
при вызове backward производные накапливаются в узлах-листьях. Так что, если backward вызывался ранее, потери оцениваются опять, backward вызывается снова (как и в любом цикле обучения), после чего накапливаются градиенты во всех листьях графа, то есть суммируются с вычисленными на предыдущей итерации, в результате чего получается неправильное значение градиента.

Чтобы предотвратить подобное, необходимо явным образом обнулять градиенты на каждой итерации. Это легко сделать с помощью метода с заменой на месте `tensor.grad.zero_`:

In [16]:
if params.grad is not None:
    params.grad.zero_()

## с учетом всего вышеописанного получим вариант GD-алгоритма с автоградом так:

In [17]:
def training_loop(n_epochs, learning_rate, params, t_u, t_c):
    for epoch in range(1, n_epochs + 1):
        
        if params.grad is not None:
            params.grad.zero_()
        
        t_p = model(t_u, params)
        loss = loss_fn(t_p, t_c)
        loss.backward()
        
        with torch.no_grad():                                  # чтобы он не дифференцировал новые параметры по старым на шаге итерации
            params -= learning_rate * params.grad

        if epoch % 500 == 0:
            print('Epoch', epoch, ': Loss =', float(loss.detach()))   # можно просто выводить loss, но некрасиво - параметр grad_fn, 
                                                                       # а при любых преобразованиях будет опять растить граф автограда
            
    return params

In [18]:
training_loop(
    n_epochs = 5000, 
    learning_rate = 1e-2, 
    params = torch.tensor([0.0, 1.0], requires_grad=True),
    t_u = t_un,
    t_c = t_c)

Epoch 500 : Loss = 7.860115051269531
Epoch 1000 : Loss = 3.828537940979004
Epoch 1500 : Loss = 3.092191219329834
Epoch 2000 : Loss = 2.957697868347168
Epoch 2500 : Loss = 2.933133840560913
Epoch 3000 : Loss = 2.9286484718322754
Epoch 3500 : Loss = 2.9278297424316406
Epoch 4000 : Loss = 2.9276793003082275
Epoch 4500 : Loss = 2.927651882171631
Epoch 5000 : Loss = 2.9276468753814697


tensor([-17.3012,   5.3671], requires_grad=True)